# KOBERT

처음엔 kobert가 단순히 문맥을 반영한 embedding을 해주는 것인줄 알았는데

찾다보니까 해결하고 싶은 task별로 활용 방법이 다르더라고요.

단순히 embedding으로 사용하는 task가 있고, transfer learning처럼 뒤에 prediction layer만 추가해서 사용하는 경우도 있더라고요.

text summarization의 경우에는 아직 어떤 방식으로 사용하는게 정답인지는 모르겠는데

지혜님이 모델 예쁘게 짜오셨으니까 그거로 학습할 수 있으면 하고 아니면 아래의 github 참고해서 하면 좋을 것 같아요. 뭔가 저희 문제에 맞는 모델인 것 같은데 tensorflow 코드를 빠르게 못 읽겠어서 일단 링크부터 공유합니다!

https://github.com/raufer/bert-summarization

https://github.com/kururuken/BERT-Transformer-for-Summarization

# 설치

pip install torch

pip install mxnet

pip install gluonnlp==0.8.0

0.9.1은 이유를 모르겠는데 설치가 안되더라고요

pip install sentencepiece

pip install onnxruntime

pip install transformers

pip install kobert-transformers

### kobert-transformers 사이트

https://pypi.org/project/kobert-transformers/

https://github.com/monologg/KoBERT-Transformers

In [1]:
import torch
import mxnet
import gluonnlp
import sentencepiece
import onnxruntime
import transformers
import kobert_transformers

C:\Users\golds\AppData\Roaming\Python\Python37\site-packages\mxnet\optimizer\optimizer.py:167: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  Optimizer.opt_registry[name].__name__))


In [2]:
print('torch : ', torch.__version__)
print('mxnet : ', mxnet.__version__)
print('gluonnlp : ', gluonnlp.__version__)
# print('sentencepiece : ', sentencepiece.__version__)
print('onnxruntime : ', onnxruntime.__version__) 
print('transformers : ', transformers.__version__)
print('kobert_transformers : ', kobert_transformers.__version__) 

torch :  1.6.0
mxnet :  1.6.0
gluonnlp :  0.8.0
onnxruntime :  1.4.0
transformers :  3.0.2
kobert_transformers :  0.4.1


In [3]:
import numpy as np
import pandas as pd
import sqlite3

# 예시로 사용할 데이터 불러오기

In [4]:
conn = sqlite3.connect('./news_cleaned.db')
cur = conn.cursor()

In [5]:
query = cur.execute('SELECT * FROM daumnews LIMIT 0, 1000')
cols = [column[0] for column in query.description]

In [6]:
daum = pd.DataFrame.from_records(data=query.fetchall(), columns=cols)

In [7]:
cur.close()
conn.close()

In [8]:
print(daum.body[0])

"벌금과 단속 걱정하던 아들, 장사 준비하던 현수막으로 목 메 숨져"결혼까지 미루고 식당 영업재개할 생각에 청소까지 열심히 했는데···"

 "아버지 아들인게 정말 행복했습니다. 이 가게가 잘 될 수 있도록 수호신이 될게요"
26살 황승우씨는 지난 30일 오전 경기 남양주시 조안면 막국수집 주방에서 목을 메 숨진 채 발견됐다. 이곳은 아버지 황선남(65)씨와 지난 2015년 희망을 품고 문을 연 가게였다.
건물주의 운전기사로 시작해 세를 얻어 희망을 품고 시작한 막국수 집은 인상이 서글서글한 부자의 웃음과 열의로 그나마 장사가 되는 듯 했다.
그러나 이곳은 팔당상수원 환경정비구역으로 남양주시와 의정부지검이 지난해 부터 단속을 벌여 음식점을 운영하던 7명이 구속됐다.
황씨가 운영하던 막국수 집도 단속을 피할 수 없었다. 지난해 12월 단속을 더이상 견딜 수 없었던 이들은 결국 음식점 문을 닫았다.
문제는 수입은 없는데 검찰이 내린 벌금 3000만원과 남양주시의 이행강제금 3690만원을 감당하기에는 너무나 힘에 부쳤다. 주변 가게에서 아르바이트를 해도 생활비는 커녕 벌금과 과태료도 낼 수 없었다.
결국 승우씨는 주변 지인과 카드빚을 내 휴가철 관광객들을 대상으로 소시지와 커피 등을 판매하는 노점상을 하기로 했다. 교제하던 여자친구와는 결혼도 미뤘다.
하지만 이마저도 한강유역환경청이 휴가철 수질을 오염시킬 우려가 있다며 지난달 집중단속을 하면서 승우씨의 노점상도 손해만 본 채 접게 됐다. 승우씨는 지인들 뿐 아니라 금융권에서도 신용불량자의 낙인이 찍힌 채 절망에 빠졌다.
이런 가운데 환경청에서 일부 원주민들에 대해서는 음식점 운영을 할 수 있도록 제도를 개선한다는 소식이 전해지면서 한줄기 희망이 생기는 듯 했다. 승우씨는 다시 장사를 할 수도 있다는 생각에 문을 닫았던 음식점의 주방부터 청소를 시작했다.
하지만 환경청이 주민들의 의견을 묵살하면서 이같은 희망도 오래가지 못했다. 승우씨는 청소를 위해 음식점을 찾았다가 이같은 소식을 접했다.

음식점 앞에는 독촉장이 

In [9]:
print(daum.summary[0])

26살 황승우씨는 지난 30일 오전 경기 남양주시 조안면 막국수집 주방에서 목을 메 숨진 채 발견됐다.
이곳은 아버지 황선남(65)씨와 지난 2015년 희망을 품고 문을 연 가게였다.
황씨가 운영하던 막국수 집도 단속을 피할 수 없었다.
하지만 이마저도 한강유역환경청이 휴가철 수질을 오염시킬 우려가 있다며 지난달 집중단속을 하면서 승우씨의 노점상도 손해만 본 채 접게 됐다.


# Tokenizing

### sktbrain-kobert 예시 코드

In [10]:
from gluonnlp.data import SentencepieceTokenizer
from kobert.utils import get_tokenizer

In [11]:
tok_path = get_tokenizer()
sp  = SentencepieceTokenizer(tok_path)

using cached model


In [12]:
example_body_token = sp(daum.body[0])
example_body_token[:20]

['▁"',
 '벌',
 '금',
 '과',
 '▁단속',
 '▁걱정',
 '하던',
 '▁아들',
 ',',
 '▁장',
 '사',
 '▁준비',
 '하던',
 '▁현',
 '수',
 '막',
 '으로',
 '▁목',
 '▁메',
 '▁숨']

In [39]:
example_summary_token = sp(daum.summary[0])
example_summary_token

['▁26',
 '살',
 '▁황',
 '승',
 '우',
 '씨는',
 '▁지난',
 '▁30',
 '일',
 '▁오전',
 '▁경기',
 '▁남',
 '양',
 '주',
 '시',
 '▁조',
 '안',
 '면',
 '▁막',
 '국',
 '수',
 '집',
 '▁주',
 '방',
 '에서',
 '▁목',
 '을',
 '▁메',
 '▁숨진',
 '▁채',
 '▁발견됐다',
 '.',
 '▁이곳',
 '은',
 '▁아버지',
 '▁황',
 '선',
 '남',
 '(',
 '65',
 ')',
 '씨와',
 '▁지난',
 '▁2015',
 '년',
 '▁희망',
 '을',
 '▁품',
 '고',
 '▁문',
 '을',
 '▁연',
 '▁',
 '가',
 '게',
 '였다',
 '.',
 '▁황',
 '씨가',
 '▁운영',
 '하던',
 '▁막',
 '국',
 '수',
 '▁집',
 '도',
 '▁단속',
 '을',
 '▁피',
 '할',
 '▁수',
 '▁없었다',
 '.',
 '▁하지만',
 '▁이',
 '마저',
 '도',
 '▁한강',
 '유',
 '역',
 '환경',
 '청',
 '이',
 '▁휴가',
 '철',
 '▁수',
 '질',
 '을',
 '▁오',
 '염',
 '시킬',
 '▁우려가',
 '▁있다',
 '며',
 '▁지난달',
 '▁집중',
 '단',
 '속',
 '을',
 '▁하면서',
 '▁승',
 '우',
 '씨의',
 '▁노',
 '점',
 '상',
 '도',
 '▁손',
 '해',
 '만',
 '▁본',
 '▁채',
 '▁접',
 '게',
 '▁됐다',
 '.']

어떻게 단어를 id로 바꾸는지 모르겠음...!

### kobert_transformers 예시 코드

In [14]:
from kobert_transformers import get_tokenizer

In [15]:
tokenizer = get_tokenizer()

In [16]:
example_body_token = tokenizer.tokenize(daum.body[0])
example_body_token[:20]

['▁"',
 '벌',
 '금',
 '과',
 '▁단속',
 '▁걱정',
 '하던',
 '▁아들',
 ',',
 '▁장',
 '사',
 '▁준비',
 '하던',
 '▁현',
 '수',
 '막',
 '으로',
 '▁목',
 '▁메',
 '▁숨']

In [17]:
example_summary_token = tokenizer.tokenize(daum.summary[0])
example_summary_token[:20]

['▁26',
 '살',
 '▁황',
 '승',
 '우',
 '씨는',
 '▁지난',
 '▁30',
 '일',
 '▁오전',
 '▁경기',
 '▁남',
 '양',
 '주',
 '시',
 '▁조',
 '안',
 '면',
 '▁막',
 '국']

In [18]:
example_body_token_id = tokenizer.convert_tokens_to_ids(example_body_token)
example_body_token_id[:20]

[518,
 6332,
 5550,
 5468,
 1590,
 880,
 7802,
 3107,
 46,
 3954,
 6493,
 4249,
 7802,
 5049,
 6629,
 6149,
 7078,
 2068,
 2016,
 2919]

In [19]:
example_summary_token_id = tokenizer.convert_tokens_to_ids(example_summary_token)
example_summary_token_id[:20]

[584,
 6519,
 5149,
 6703,
 7005,
 6787,
 4304,
 592,
 7126,
 3431,
 956,
 1409,
 6853,
 7276,
 6705,
 4162,
 6812,
 6198,
 1927,
 5503]

# Embedding

### sktbrain-kobert 예시 코드

from kobert.pytorch_kobert import get_pytorch_kobert_model

model, vocab  = get_pytorch_kobert_model()

### kobert_transformers 예시 코드

In [20]:
from kobert_transformers import get_kobert_model

In [28]:
model = get_kobert_model()
# model.eval()

In [29]:
example_body_tensor = torch.LongTensor([example_body_token_id])
example_summary_tensor = torch.LongTensor([example_summary_token_id])

In [30]:
example_body_tensor.shape

torch.Size([1, 883])

In [31]:
try:
    body_sequence_output, body_pooled_output = model(example_body_tensor)
except IndexError as e:
    print(e)

index out of range in self


embedding vector의 차원이 768이니까 input의 길이가 768보다 커서 index error가 뜨는것인가...?

단어 개수가 768개보다 작은거만 input으로 넣을 수 있다는게 말이 안되지 않나...?

In [32]:
summary_sequence_output, summary_pooled_output = model(example_summary_tensor)

In [33]:
example_summary_tensor.shape

torch.Size([1, 116])

In [34]:
summary_sequence_output.shape, summary_pooled_output.shape

(torch.Size([1, 116, 768]), torch.Size([1, 768]))

1개의 sample

단어수가 116개

embedding vector의 차원이 768

문장의 시작에 [CLS], 문장의 끝에 [SEP]를 무조건 넣어줘야하나??

BERT has a constraint on the maximum length of a sequence after tokenizing. For any BERT model, the maximum sequence length after tokenization is 512. But we can set any sequence length equal to or below this value. For faster training, I’ll be using 128 as the maximum sequence length. A bigger number may give better results if there are sequences longer than this value.

토큰화를 한 후 sequence의 최대 길이는 512

글의 시작에 [CLS]를 적어주니까 사용할 수 있는 토큰 개수는 511

embedding만 해주는게 아니고

embedding + pre-trained modeling이라서 저희가 짜야하는 모델은 그 뒤에 붙혀서 학습할 모델이 되야하는건지??